In [2]:
import pandas as pd
import os
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

# --- 0. Carregamento dos Datasets ---

In [9]:
# Define o caminho para a pasta 'raw' dos dados
# O '../' é para subir um nível do diretório 'notebooks' para o diretório raiz do projeto,
# e então descer para 'data/raw'
RAW_DATA_PATH = '../data/raw/'

# Lista dos arquivos CSV
csv_files = {
    'accounts': 'accounts.csv',
    'data_dictionary': 'data_dictionary.csv',
    'products': 'products.csv',
    'sales_pipeline': 'sales_pipeline_simulated_v3.csv',
    'sales_teams': 'sales_teams.csv'
}

# Dicionário para armazenar os DataFrames
dfs = {}

# Carrega cada arquivo CSV em um DataFrame e armazena no dicionário
for name, file_name in csv_files.items():
    file_path = os.path.join(RAW_DATA_PATH, file_name)
    try:
        dfs[name] = pd.read_csv(file_path)
        print(f"DataFrame '{name}' carregado com sucesso! Shape: {dfs[name].shape}")
    except FileNotFoundError:
        print(f"Erro: Arquivo '{file_name}' não encontrado em {file_path}")
    except Exception as e:
        print(f"Erro ao carregar '{file_name}': {e}")

print("\n--- Carregamento de dados concluído ---")

DataFrame 'accounts' carregado com sucesso! Shape: (85, 7)
DataFrame 'data_dictionary' carregado com sucesso! Shape: (21, 3)
DataFrame 'products' carregado com sucesso! Shape: (7, 3)
DataFrame 'sales_pipeline' carregado com sucesso! Shape: (8800, 9)
DataFrame 'sales_teams' carregado com sucesso! Shape: (35, 3)

--- Carregamento de dados concluído ---


In [10]:
print("\n--- Primeiras linhas de sales_pipeline ---")
print(dfs['sales_pipeline'].head())


--- Primeiras linhas de sales_pipeline ---
  opportunity_id      sales_agent         product  account   deal_stage  \
0       1C1I7A6R      Moses Frase  GTX Plus Basic  Cancity     Engaging   
1       Z063OYW0  Darcel Schlecht          GTXPro    Isdom  Prospecting   
2       EC4QE1BX  Darcel Schlecht      MG Special  Cancity     Engaging   
3       MV1LWRNH      Moses Frase       GTX Basic  Codehow     Engaging   
4       PE84CX4O        Zane Levy       GTX Basic   Hatfan     Engaging   

  engage_date  close_date  close_value status  
0  2016-10-20  2017-03-01       1054.0    Won  
1  2016-10-25  2017-03-11       4514.0    Won  
2  2016-10-25  2017-03-07         50.0    Won  
3  2016-10-25  2017-03-09        588.0    Won  
4  2016-10-25  2017-03-02        517.0    Won  


In [11]:
print("\n--- Informações de sales_pipeline ---")
dfs['sales_pipeline'].info()


--- Informações de sales_pipeline ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8800 entries, 0 to 8799
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   opportunity_id  8800 non-null   object 
 1   sales_agent     8800 non-null   object 
 2   product         8800 non-null   object 
 3   account         7375 non-null   object 
 4   deal_stage      8800 non-null   object 
 5   engage_date     8300 non-null   object 
 6   close_date      6711 non-null   object 
 7   close_value     7487 non-null   float64
 8   status          8800 non-null   object 
dtypes: float64(1), object(8)
memory usage: 618.9+ KB


In [12]:
print("\n--- Estatísticas descritivas de sales_pipeline ---")
print(dfs['sales_pipeline'].describe())


--- Estatísticas descritivas de sales_pipeline ---
        close_value
count   7487.000000
mean    2388.307508
std     2611.303774
min       38.000000
25%      545.642623
50%     1080.053599
75%     4606.000000
max    30288.000000


In [13]:
print("\n--- Contagem de valores para a coluna 'deal_stage' ---")
print(dfs['sales_pipeline']['deal_stage'].value_counts())


--- Contagem de valores para a coluna 'deal_stage' ---
deal_stage
Engaging       6713
Prospecting    2087
Name: count, dtype: int64


# --- 1. Criação da Variável Alvo (Target) ---

In [14]:
# A coluna 'status' é a nossa variável alvo com 'Won' e 'Lost'.
# Criar uma cópia para não modificar o DataFrame original diretamente antes de explorar
df_sales_pipeline_eda = dfs['sales_pipeline'].copy()

# Vamos mapeá-la para valores numéricos: 0 para 'Won' (Não Perdida) e 1 para 'Lost' (Perdida).
df_sales_pipeline_eda['target'] = df_sales_pipeline_eda['status'].map({'Won': 0, 'Lost': 1})

# Remover a coluna 'status' original, pois 'target' agora a representa numericamente.
# É importante remover 'status' do DataFrame para que não seja tratada como feature posteriormente.
df_sales_pipeline_eda = df_sales_pipeline_eda.drop('status', axis=1)

print("\nContagem da Variável Alvo 'target':")
print(df_sales_pipeline_eda['target'].value_counts())
print(f"Proporção de oportunidades perdidas (1): {df_sales_pipeline_eda['target'].value_counts(normalize=True)[1]:.2%}")



Contagem da Variável Alvo 'target':
target
0    5551
1    3249
Name: count, dtype: int64
Proporção de oportunidades perdidas (1): 36.92%


# --- 2. Conversão de Tipos de Dados (Datas) ---

In [15]:
# --- 2. Conversão de Tipos de Dados (Datas) ---
# Converter colunas de data para datetime
df_sales_pipeline_eda['engage_date'] = pd.to_datetime(df_sales_pipeline_eda['engage_date'], errors='coerce')
df_sales_pipeline_eda['close_date'] = pd.to_datetime(df_sales_pipeline_eda['close_date'], errors='coerce')

print("\nTipos de dados após conversão de datas:")
df_sales_pipeline_eda.info()


Tipos de dados após conversão de datas:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8800 entries, 0 to 8799
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   opportunity_id  8800 non-null   object        
 1   sales_agent     8800 non-null   object        
 2   product         8800 non-null   object        
 3   account         7375 non-null   object        
 4   deal_stage      8800 non-null   object        
 5   engage_date     8300 non-null   datetime64[ns]
 6   close_date      6711 non-null   datetime64[ns]
 7   close_value     7487 non-null   float64       
 8   target          8800 non-null   int64         
dtypes: datetime64[ns](2), float64(1), int64(1), object(5)
memory usage: 618.9+ KB


# --- 3. Plot: Distribuição da Variável Alvo ---

In [19]:
# Captura o resultado do value_counts e reseta o índice, nomeando as colunas
target_counts = df_sales_pipeline_eda['target'].value_counts(normalize=True).reset_index()
target_counts.columns = ['target_status', 'proportion'] # Renomeando as colunas para clareza

fig = px.bar(target_counts,
             x='target_status', y='proportion',
             title='Distribuição da Variável Alvo (Oportunidades Perdidas vs. Não Perdidas)',
             labels={'target_status': 'Status da Oportunidade', 'proportion': 'Proporção'},
             color='target_status',
             color_discrete_map={0: 'lightgreen', 1: 'salmon'},
             text_auto='.2%') # Exibir o percentual direto na barra

fig.update_layout(xaxis_title="0: Não Perdida | 1: Perdida",
                  yaxis_title="Proporção")
fig.show()

In [20]:
dfs['data_dictionary'].head()

,Table,Field,Description
0,accounts,account,Company name
1,accounts,sector,Industry
2,accounts,year_established,Year Established
3,accounts,revenue,Annual revenue (in millions of USD)
4,accounts,employees,Number of employees


In [21]:
dfs['data_dictionary'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Table        21 non-null     object
 1   Field        21 non-null     object
 2   Description  21 non-null     object
dtypes: object(3)
memory usage: 636.0+ bytes


In [22]:
dfs['data_dictionary'].describe()

,Table,Field,Description
count,21,21,21
unique,4,18,19
top,sales_pipeline,account,Company name
freq,8,2,2


# --- 4. Análise RFM (Recência, Frequência, Valor Monetário) ---

In [23]:
print("\n--- Iniciando Análise RFM ---")


--- Iniciando Análise RFM ---


## 4.1. Filtrar apenas oportunidades GANHAS ('Won')

In [61]:
# A análise RFM foca em clientes que tiveram o target como 0 (ganhas).
df_won_opportunities = df_sales_pipeline_eda[df_sales_pipeline_eda['target'] == 0].copy()

# Remover linhas onde close_date ou close_value são nulos para RFM, pois não são transações completas
# mas ainda precisamos de valores válidos e > 0 para as oportunidades 'Won'.
df_won_opportunities.dropna(subset=['close_date', 'close_value', 'account'], inplace=True)

# Garantir que close_value é numérico e > 0 para RFM
df_won_opportunities['close_value'] = pd.to_numeric(df_won_opportunities['close_value'], errors='coerce')
df_won_opportunities = df_won_opportunities[df_won_opportunities['close_value'] > 0]


if not df_won_opportunities.empty:
    # 4.2. Calcular Recência, Frequência, Valor Monetário
    # A data de referência é a data mais recente de fechamento no dataset
    snapshot_date = df_won_opportunities['close_date'].max() + pd.Timedelta(days=1) # Um dia após a última venda

    rfm_df = df_won_opportunities.groupby('account').agg(
        Recency=('close_date', lambda date: (snapshot_date - date.max()).days),
        Frequency=('opportunity_id', 'count'),
        Monetary=('close_value', 'sum')
    ).reset_index()

    print("\n--- DataFrame RFM (primeiras 5 linhas) ---")
    print(rfm_df.head())

    print("\n--- Estatísticas descritivas do DataFrame RFM ---")
    print(rfm_df.describe())

    # 4.3. Visualização RFM (Exemplo: Histogramas das distribuições)
    fig = make_subplots(rows=1, cols=3, subplot_titles=("Recência (Dias)", "Frequência (Oportunidades Ganhas)", "Valor Monetário Total"))

    fig.add_trace(go.Histogram(x=rfm_df['Recency'], name='Recência'), row=1, col=1)
    fig.add_trace(go.Histogram(x=rfm_df['Frequency'], name='Frequência'), row=1, col=2)
    fig.add_trace(go.Histogram(x=rfm_df['Monetary'], name='Monetário'), row=1, col=3)

    fig.update_layout(title_text='Distribuição das Métricas RFM', height=400, showlegend=False)
    fig.show()

    # Segmentação RFM básica (ex: quintis)
    rfm_df['R_score'] = pd.qcut(rfm_df['Recency'], 5, labels=False, duplicates='drop')
    rfm_df['F_score'] = pd.qcut(rfm_df['Frequency'], 5, labels=False, duplicates='drop')
    rfm_df['M_score'] = pd.qcut(rfm_df['Monetary'], 5, labels=False, duplicates='drop')
    print("\n--- RFM Scores (primeiras 5 linhas) ---")
    print(rfm_df.head())

else:
    print("Não há oportunidades 'Ganhas' (target=0) suficientes para realizar a análise RFM após filtragem de NaNs ou close_value <= 0.")



--- DataFrame RFM (primeiras 5 linhas) ---
            account  Recency  Frequency  Monetary
0  Acme Corporation        5         34  101744.0
1        Betasoloin        4         34   97036.0
2          Betatech        3         53  107408.0
3        Bioholding        4         50   90991.0
4           Bioplex       16         31   67393.0

--- Estatísticas descritivas do DataFrame RFM ---
         Recency   Frequency       Monetary
count  85.000000   85.000000      85.000000
mean    5.129412   49.858824  117712.164706
std     5.098525   18.538347   46525.656119
min     1.000000   23.000000   51632.000000
25%     2.000000   36.000000   85047.000000
50%     4.000000   47.000000  111533.000000
75%     7.000000   57.000000  140086.000000
max    33.000000  115.000000  341455.000000



--- RFM Scores (primeiras 5 linhas) ---
            account  Recency  Frequency  Monetary  R_score  F_score  M_score
0  Acme Corporation        5         34  101744.0        2        0        2
1        Betasoloin        4         34   97036.0        1        0        1
2          Betatech        3         53  107408.0        1        3        2
3        Bioholding        4         50   90991.0        1        2        1
4           Bioplex       16         31   67393.0        3        0        0


# --- Análise do DataFrame 'accounts' ---

In [ ]:
print("\n--- Primeiras linhas de accounts ---")
print(dfs['accounts'].head())


--- Primeiras linhas de accounts ---
            account     sector  year_established  revenue  employees  \
0  Acme Corporation  technolgy              1996  1100.04       2822   
1        Betasoloin    medical              1999   251.41        495   
2          Betatech    medical              1986   647.18       1185   
3        Bioholding    medical              2012   587.34       1356   
4           Bioplex    medical              1991   326.82       1016   

  office_location subsidiary_of  
0   United States           NaN  
1   United States           NaN  
2           Kenya           NaN  
3      Philipines           NaN  
4   United States           NaN  


In [101]:
print("\n--- Informações de accounts ---")
dfs['accounts'].info()


--- Informações de accounts ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85 entries, 0 to 84
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   account           85 non-null     object 
 1   sector            85 non-null     object 
 2   year_established  85 non-null     int64  
 3   revenue           85 non-null     float64
 4   employees         85 non-null     int64  
 5   office_location   85 non-null     object 
 6   subsidiary_of     15 non-null     object 
dtypes: float64(1), int64(2), object(4)
memory usage: 4.8+ KB


In [102]:
print("\n--- Estatísticas descritivas de accounts ---")
print(dfs['accounts'].describe())


--- Estatísticas descritivas de accounts ---
       year_established       revenue     employees
count         85.000000     85.000000     85.000000
mean        1996.105882   1994.632941   4660.823529
std            8.865427   2169.491436   5715.601198
min         1979.000000      4.540000      9.000000
25%         1989.000000    497.110000   1179.000000
50%         1996.000000   1223.720000   2769.000000
75%         2002.000000   2741.370000   5595.000000
max         2017.000000  11698.030000  34288.000000


In [103]:
print("\n--- Contagem de valores para 'sector' em accounts ---")
print(dfs['accounts']['sector'].value_counts())


--- Contagem de valores para 'sector' em accounts ---
sector
retail                17
technolgy             12
medical               12
marketing              8
finance                8
software               7
entertainment          6
telecommunications     6
services               5
employment             4
Name: count, dtype: int64


In [104]:
print("\n--- Contagem de valores para 'year_established' em accounts (Top 10) ---")
print(dfs['accounts']['year_established'].value_counts().head(10))


--- Contagem de valores para 'year_established' em accounts (Top 10) ---
year_established
1996    4
1995    4
1993    4
1997    4
1994    4
2000    3
1999    3
1992    3
1982    3
1998    3
Name: count, dtype: int64


In [105]:
print("\n--- Estatísticas descritivas de 'revenue' e 'employees' em accounts ---")
print(dfs['accounts'][['revenue', 'employees']].describe())


--- Estatísticas descritivas de 'revenue' e 'employees' em accounts ---
            revenue     employees
count     85.000000     85.000000
mean    1994.632941   4660.823529
std     2169.491436   5715.601198
min        4.540000      9.000000
25%      497.110000   1179.000000
50%     1223.720000   2769.000000
75%     2741.370000   5595.000000
max    11698.030000  34288.000000


# --- Análise do DataFrame 'sales_teams' ---


In [106]:
print("\n--- Primeiras linhas de sales_teams ---")
print(dfs['sales_teams'].head())


--- Primeiras linhas de sales_teams ---
         sales_agent           manager regional_office
0      Anna Snelling  Dustin Brinkmann         Central
1     Cecily Lampkin  Dustin Brinkmann         Central
2  Versie Hillebrand  Dustin Brinkmann         Central
3    Lajuana Vencill  Dustin Brinkmann         Central
4        Moses Frase  Dustin Brinkmann         Central


In [107]:
print("\n--- Informações de sales_teams ---")
dfs['sales_teams'].info()


--- Informações de sales_teams ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   sales_agent      35 non-null     object
 1   manager          35 non-null     object
 2   regional_office  35 non-null     object
dtypes: object(3)
memory usage: 972.0+ bytes


In [108]:
print("\n--- Estatísticas descritivas de sales_teams ---")
print(dfs['sales_teams'].describe())


--- Estatísticas descritivas de sales_teams ---
          sales_agent        manager regional_office
count              35             35              35
unique             35              6               3
top     Anna Snelling  Melvin Marxen            East
freq                1              6              12


In [109]:
print("\n--- Contagem de valores para 'sales_agent' em sales_teams ---")
print(dfs['sales_teams']['sales_agent'].value_counts()) # Ou similar, dependendo do nome da coluna do agente


--- Contagem de valores para 'sales_agent' em sales_teams ---
sales_agent
Anna Snelling         1
Cecily Lampkin        1
Versie Hillebrand     1
Lajuana Vencill       1
Moses Frase           1
Jonathan Berthelot    1
Marty Freudenburg     1
Gladys Colclough      1
Niesha Huffines       1
Darcel Schlecht       1
Mei-Mei Johns         1
Violet Mclelland      1
Corliss Cosme         1
Rosie Papadopoulos    1
Garret Kinder         1
Wilburn Farren        1
Elizabeth Anderson    1
Daniell Hammack       1
Cassey Cress          1
Donn Cantrell         1
Reed Clapper          1
Boris Faz             1
Natalya Ivanova       1
Vicki Laflamme        1
Rosalina Dieter       1
Hayden Neloms         1
Markita Hansen        1
Elease Gluck          1
Carol Thompson        1
James Ascencio        1
Kary Hendrixson       1
Kami Bicknell         1
Zane Levy             1
Maureen Marcano       1
Carl Lin              1
Name: count, dtype: int64


# --- Análise do DataFrame 'products' ---

In [111]:
print("\n--- Primeiras linhas de products ---")
print(dfs['products'].head())


--- Primeiras linhas de products ---
        product series  sales_price
0     GTX Basic    GTX          550
1       GTX Pro    GTX         4821
2    MG Special     MG           55
3   MG Advanced     MG         3393
4  GTX Plus Pro    GTX         5482


In [112]:
print("\n--- Informações de products ---")
dfs['products'].info()


--- Informações de products ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   product      7 non-null      object
 1   series       7 non-null      object
 2   sales_price  7 non-null      int64 
dtypes: int64(1), object(2)
memory usage: 300.0+ bytes


In [113]:
print("\n--- Estatísticas descritivas de products ---")
print(dfs['products'].describe())


--- Estatísticas descritivas de products ---
        sales_price
count      7.000000
mean    6023.571429
std     9388.428070
min       55.000000
25%      823.000000
50%     3393.000000
75%     5151.500000
max    26768.000000


In [114]:
print("\n--- Contagem de valores para 'product' em products ---")
print(dfs['products']['product'].value_counts()) # Ou similar, dependendo do nome da coluna de produto


--- Contagem de valores para 'product' em products ---
product
GTX Basic         1
GTX Pro           1
MG Special        1
MG Advanced       1
GTX Plus Pro      1
GTX Plus Basic    1
GTK 500           1
Name: count, dtype: int64


# --- 5. Unificação dos DataFrames (Merge) ---

In [115]:
print("\n--- Unificando DataFrames para EDA aprofundada ---")

# Criar uma cópia do DataFrame principal (sales_pipeline) para o merge
df_eda_consolidated = df_sales_pipeline_eda.copy()


--- Unificando DataFrames para EDA aprofundada ---


In [116]:
# Merge com 'accounts'
df_eda_consolidated = pd.merge(df_eda_consolidated, dfs['accounts'], on='account', how='left')

# Merge com 'products'
df_eda_consolidated = pd.merge(df_eda_consolidated, dfs['products'], on='product', how='left')

# Merge com 'sales_teams'
df_eda_consolidated = pd.merge(df_eda_consolidated, dfs['sales_teams'], on='sales_agent', how='left')

In [117]:
print("\n--- DataFrame Consolidado para EDA (primeiras 5 linhas) ---")
print(df_eda_consolidated.head())


--- DataFrame Consolidado para EDA (primeiras 5 linhas) ---
  opportunity_id      sales_agent         product  account   deal_stage  \
0       1C1I7A6R      Moses Frase  GTX Plus Basic  Cancity     Engaging   
1       Z063OYW0  Darcel Schlecht          GTXPro    Isdom  Prospecting   
2       EC4QE1BX  Darcel Schlecht      MG Special  Cancity     Engaging   
3       MV1LWRNH      Moses Frase       GTX Basic  Codehow     Engaging   
4       PE84CX4O        Zane Levy       GTX Basic   Hatfan     Engaging   

  engage_date close_date  close_value  target    sector  year_established  \
0  2016-10-20 2017-03-01       1054.0       0    retail            2001.0   
1  2016-10-25 2017-03-11       4514.0       0   medical            2002.0   
2  2016-10-25 2017-03-07         50.0       0    retail            2001.0   
3  2016-10-25 2017-03-09        588.0       0  software            1998.0   
4  2016-10-25 2017-03-02        517.0       0  services            1982.0   

   revenue  employees off

In [118]:
print("\n--- Informações do DataFrame Consolidado ---")
df_eda_consolidated.info()


--- Informações do DataFrame Consolidado ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8800 entries, 0 to 8799
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   opportunity_id    8800 non-null   object        
 1   sales_agent       8800 non-null   object        
 2   product           8800 non-null   object        
 3   account           7375 non-null   object        
 4   deal_stage        8800 non-null   object        
 5   engage_date       8300 non-null   datetime64[ns]
 6   close_date        6711 non-null   datetime64[ns]
 7   close_value       7487 non-null   float64       
 8   target            8800 non-null   int64         
 9   sector            7375 non-null   object        
 10  year_established  7375 non-null   float64       
 11  revenue           7375 non-null   float64       
 12  employees         7375 non-null   float64       
 13  office_location   7375 non-null 

In [119]:
# Verificar se há duplicatas após o merge
print(f"\nNúmero de duplicatas após merge: {df_eda_consolidated.duplicated().sum()}")



Número de duplicatas após merge: 0


# --- Análise por Setor (Sector) ---

In [120]:
print("\n--- Análise da Taxa de Perda por Setor ---")

# Calcular a contagem de oportunidades por setor e a contagem de oportunidades perdidas por setor
sector_analysis = df_eda_consolidated.groupby('sector')['target'].agg(
    total_opportunities='count',
    lost_opportunities=lambda x: (x == 1).sum()
).reset_index()

# Calcular a taxa de perda por setor
sector_analysis['loss_rate'] = (sector_analysis['lost_opportunities'] / sector_analysis['total_opportunities']) * 100

# Ordenar para visualização
sector_analysis = sector_analysis.sort_values(by='loss_rate', ascending=False)

print(sector_analysis)


--- Análise da Taxa de Perda por Setor ---
               sector  total_opportunities  lost_opportunities  loss_rate
2             finance                  678                 261  38.495575
4             medical                 1051                 404  38.439581
9  telecommunications                  501                 190  37.924152
0          employment                  311                 117  37.620579
5              retail                 1397                 523  37.437366
6            services                  390                 144  36.923077
7            software                  757                 275  36.327609
8           technolgy                 1165                 419  35.965665
1       entertainment                  451                 162  35.920177
3           marketing                  674                 234  34.718101


In [121]:
# Plot com Plotly
fig = px.bar(sector_analysis,
             x='sector',
             y='loss_rate',
             title='Taxa de Perda de Oportunidades por Setor de Cliente',
             labels={'sector': 'Setor do Cliente', 'loss_rate': 'Taxa de Perda (%)'},
             color='loss_rate', # Colorir as barras pela taxa de perda
             color_continuous_scale=px.colors.sequential.Reds, # Escala de cores de vermelho para destacar taxas altas
             text_auto='.2f') # Exibir o valor da taxa de perda nas barras

fig.update_layout(xaxis_title="Setor",
                  yaxis_title="Taxa de Perda (%)",
                  xaxis_tickangle=-45) # Rotacionar os rótulos do eixo X para melhor leitura
fig.show()

# --- Análise por Porte da Empresa (Revenue e Employees) ---

In [122]:
print("\n--- Análise da Relação entre Porte da Empresa e Perda de Oportunidade ---")

# Remover NaNs das colunas relevantes para esta análise, se for o caso
df_filtered_for_size_analysis = df_eda_consolidated.dropna(subset=['revenue', 'employees', 'target']).copy()

# Scatter plot: Revenue vs. Loss (com cor para indicar se foi perdida ou não)
fig1 = px.scatter(df_filtered_for_size_analysis,
                  x='revenue',
                  y='employees',
                  color='target',
                  hover_data=['account', 'deal_stage', 'close_value'],
                  title='Faturamento (Revenue) vs. Número de Funcionários (Employees) por Desfecho da Oportunidade',
                  labels={'revenue': 'Faturamento Anual (Milhões USD)', 'employees': 'Número de Funcionários'},
                  color_discrete_map={0: 'lightgreen', 1: 'salmon'},
                  log_x=True, log_y=True) # Usar escala logarítmica devido à grande variação

fig1.update_layout(xaxis_title="Faturamento Anual (Milhões USD) [Escala Logarítmica]",
                   yaxis_title="Número de Funcionários [Escala Logarítmica]")
fig1.show()

# Box Plot: Comparação de Revenue para Oportunidades Perdidas vs. Não Perdidas
fig2 = px.box(df_filtered_for_size_analysis,
              x='target',
              y='revenue',
              color='target',
              title='Distribuição de Faturamento para Oportunidades Perdidas vs. Não Perdidas',
              labels={'target': 'Status da Oportunidade', 'revenue': 'Faturamento Anual (Milhões USD)'},
              color_discrete_map={0: 'lightgreen', 1: 'salmon'})
fig2.update_layout(xaxis_title="0: Não Perdida | 1: Perdida")
fig2.show()

# Box Plot: Comparação de Employees para Oportunidades Perdidas vs. Não Perdidas
fig3 = px.box(df_filtered_for_size_analysis,
              x='target',
              y='employees',
              color='target',
              title='Distribuição de Número de Funcionários para Oportunidades Perdidas vs. Não Perdidas',
              labels={'target': 'Status da Oportunidade', 'employees': 'Número de Funcionários'},
              color_discrete_map={0: 'lightgreen', 1: 'salmon'})
fig3.update_layout(xaxis_title="0: Não Perdida | 1: Perdida")
fig3.show()



--- Análise da Relação entre Porte da Empresa e Perda de Oportunidade ---


# --- Análise por Quintis de Revenue e Employees ---

In [125]:
# Remover NaNs das colunas relevantes para esta análise
df_temp_analysis = df_eda_consolidated.dropna(subset=['revenue', 'employees', 'target']).copy()

# Criar quintis para Revenue
df_temp_analysis['revenue_quintile'] = pd.qcut(df_temp_analysis['revenue'], 5, labels=False, duplicates='drop')

# Calcular taxa de perda por quintil de Revenue
revenue_loss_rate = df_temp_analysis.groupby('revenue_quintile')['target'].agg(
    total_opportunities='count',
    lost_opportunities=lambda x: (x == 1).sum(),
    loss_rate=lambda x: (x == 1).sum() / x.count() * 100
).reset_index()

print("\n--- Taxa de Perda por Quintil de Faturamento (Revenue) ---")
print(revenue_loss_rate)

# Criar quintis para Employees
df_temp_analysis['employees_quintile'] = pd.qcut(df_temp_analysis['employees'], 5, labels=False, duplicates='drop')

# Calcular taxa de perda por quintil de Employees
employees_loss_rate = df_temp_analysis.groupby('employees_quintile')['target'].agg(
    total_opportunities='count',
    lost_opportunities=lambda x: (x == 1).sum(),
    loss_rate=lambda x: (x == 1).sum() / x.count() * 100
).reset_index()

print("\n--- Taxa de Perda por Quintil de Número de Funcionários (Employees) ---")
print(employees_loss_rate)


--- Taxa de Perda por Quintil de Faturamento (Revenue) ---
   revenue_quintile  total_opportunities  lost_opportunities  loss_rate
0                 0                 1556                 555  35.668380
1                 1                 1447                 561  38.769869
2                 2                 1515                 547  36.105611
3                 3                 1437                 521  36.256089
4                 4                 1420                 545  38.380282

--- Taxa de Perda por Quintil de Número de Funcionários (Employees) ---
   employees_quintile  total_opportunities  lost_opportunities  loss_rate
0                   0                 1512                 549  36.309524
1                   1                 1446                 551  38.105118
2                   2                 1481                 547  36.934504
3                   3                 1554                 553  35.585586
4                   4                 1382                 529  3

# --- Análise da Taxa de Perda por Produto ---

In [126]:
print("\n--- Análise da Taxa de Perda por Produto ---")

# Calcular a contagem de oportunidades por produto e a contagem de oportunidades perdidas por produto
product_analysis = df_eda_consolidated.groupby('product')['target'].agg(
    total_opportunities='count',
    lost_opportunities=lambda x: (x == 1).sum()
).reset_index()

# Calcular a taxa de perda por produto
product_analysis['loss_rate'] = (product_analysis['lost_opportunities'] / product_analysis['total_opportunities']) * 100

# Ordenar para visualização (opcional)
product_analysis = product_analysis.sort_values(by='loss_rate', ascending=False)

print(product_analysis)

# Plot com Plotly: Taxa de Perda por Produto
fig_product = px.bar(product_analysis,
                     x='product',
                     y='loss_rate',
                     title='Taxa de Perda de Oportunidades por Produto',
                     labels={'product': 'Produto', 'loss_rate': 'Taxa de Perda (%)'},
                     color='loss_rate',
                     color_continuous_scale=px.colors.sequential.YlOrRd, # Destacar taxas mais altas
                     text_auto='.2f') # Exibir o valor da taxa de perda

fig_product.update_layout(xaxis_title="Produto",
                          yaxis_title="Taxa de Perda (%)",
                          xaxis_tickangle=-45)
fig_product.show()


--- Análise da Taxa de Perda por Produto ---
          product  total_opportunities  lost_opportunities  loss_rate
0         GTK 500                   40                  17  42.500000
5     MG Advanced                 1412                 554  39.235127
2  GTX Plus Basic                 1383                 526  38.033261
3    GTX Plus Pro                  968                 357  36.880165
4          GTXPro                 1480                 545  36.824324
6      MG Special                 1651                 592  35.857056
1       GTX Basic                 1866                 658  35.262594


# --- Análise da Taxa de Perda por Série de Produto ---

In [127]:
print("\n--- Análise da Taxa de Perda por Série de Produto ---")

# Calcular a contagem de oportunidades por série e a contagem de oportunidades perdidas por série
series_analysis = df_eda_consolidated.groupby('series')['target'].agg(
    total_opportunities='count',
    lost_opportunities=lambda x: (x == 1).sum()
).reset_index()

# Calcular a taxa de perda por série
series_analysis['loss_rate'] = (series_analysis['lost_opportunities'] / series_analysis['total_opportunities']) * 100

# Ordenar para visualização (opcional)
series_analysis = series_analysis.sort_values(by='loss_rate', ascending=False)

print(series_analysis)

# Plot com Plotly: Taxa de Perda por Série de Produto
fig_series = px.bar(series_analysis,
                    x='series',
                    y='loss_rate',
                    title='Taxa de Perda de Oportunidades por Série de Produto',
                    labels={'series': 'Série do Produto', 'loss_rate': 'Taxa de Perda (%)'},
                    color='loss_rate',
                    color_continuous_scale=px.colors.sequential.YlOrRd,
                    text_auto='.2f')

fig_series.update_layout(xaxis_title="Série do Produto",
                         yaxis_title="Taxa de Perda (%)")
fig_series.show()


--- Análise da Taxa de Perda por Série de Produto ---
  series  total_opportunities  lost_opportunities  loss_rate
0    GTK                   40                  17  42.500000
2     MG                 3063                1146  37.414300
1    GTX                 4217                1541  36.542566


# --- Análise da Relação entre Preço de Venda e Perda de Oportunidade ---

In [128]:
print("\n--- Análise da Relação entre Preço de Venda e Perda de Oportunidade ---")

# Remover NaNs das colunas relevantes para esta análise
df_filtered_for_price_analysis = df_eda_consolidated.dropna(subset=['sales_price', 'target']).copy()

# Box Plot: Comparação de sales_price para Oportunidades Perdidas vs. Não Perdidas
fig_price = px.box(df_filtered_for_price_analysis,
                   x='target',
                   y='sales_price',
                   color='target',
                   title='Distribuição do Preço de Venda para Oportunidades Perdidas vs. Não Perdidas',
                   labels={'target': 'Status da Oportunidade', 'sales_price': 'Preço de Venda'},
                   color_discrete_map={0: 'lightgreen', 1: 'salmon'})

fig_price.update_layout(xaxis_title="0: Não Perdida | 1: Perdida")
fig_price.show()


--- Análise da Relação entre Preço de Venda e Perda de Oportunidade ---


In [129]:
# Para uma análise mais granular, pode-se criar faixas de preço e calcular a taxa de perda
df_filtered_for_price_analysis['price_bin'] = pd.qcut(df_filtered_for_price_analysis['sales_price'], 5, labels=False, duplicates='drop')
price_bin_loss_rate = df_filtered_for_price_analysis.groupby('price_bin')['target'].agg(
     total_opportunities='count',
     lost_opportunities=lambda x: (x == 1).sum(),
     loss_rate=lambda x: (x == 1).sum() / x.count() * 100
 ).reset_index()
print("\n--- Taxa de Perda por Quintil de Preço de Venda ---")
print(price_bin_loss_rate)


--- Taxa de Perda por Quintil de Preço de Venda ---
   price_bin  total_opportunities  lost_opportunities  loss_rate
0          0                 3517                1250  35.541655
1          1                 1383                 526  38.033261
2          2                 1412                 554  39.235127
3          3                 1008                 374  37.103175


# --- Análise da Taxa de Perda por Agente de Vendas ---

In [131]:
print("\n--- Análise da Taxa de Perda por Agente de Vendas ---")

# Calcular a contagem de oportunidades por agente e a contagem de oportunidades perdidas por agente
agent_analysis = df_eda_consolidated.groupby('sales_agent')['target'].agg(
    total_opportunities='count',
    lost_opportunities=lambda x: (x == 1).sum()
).reset_index()

# Calcular a taxa de perda por agente
agent_analysis['loss_rate'] = (agent_analysis['lost_opportunities'] / agent_analysis['total_opportunities']) * 100

# Ordenar para visualização (top/bottom agentes)
agent_analysis = agent_analysis.sort_values(by='loss_rate', ascending=False)

print("Agentes com as 5 Maiores Taxas de Perda:")
print(agent_analysis.head())

print("\nAgentes com as 5 Menores Taxas de Perda:")
print(agent_analysis.tail())

# Plot com Plotly: Top/Bottom Agentes por Taxa de Perda
# Como há muitos agentes, focaremos nos extremos para o gráfico.
# Ex: Top 10 e Bottom 10
top_agents = agent_analysis.head(10)
bottom_agents = agent_analysis.tail(10)
agents_to_plot = pd.concat([top_agents, bottom_agents]).sort_values(by='loss_rate', ascending=False)


fig_agent = px.bar(agents_to_plot,
                   x='sales_agent',
                   y='loss_rate',
                   title='Top/Bottom Agentes de Vendas por Taxa de Perda de Oportunidades',
                   labels={'sales_agent': 'Agente de Vendas', 'loss_rate': 'Taxa de Perda (%)'},
                   color='loss_rate',
                   color_continuous_scale="RdYlGn_r",
                   text_auto='.2f')

fig_agent.update_layout(xaxis_title="Agente de Vendas",
                        yaxis_title="Taxa de Perda (%)",
                        xaxis_tickangle=-45)
fig_agent.show()


--- Análise da Taxa de Perda por Agente de Vendas ---
Agentes com as 5 Maiores Taxas de Perda:
         sales_agent  total_opportunities  lost_opportunities  loss_rate
16   Lajuana Vencill                  311                 134  43.086817
10  Gladys Colclough                  317                 135  42.586751
7      Donn Cantrell                  275                 117  42.545455
17    Markita Hansen                  306                 124  40.522876
21   Niesha Huffines                  239                  94  39.330544

Agentes com as 5 Menores Taxas de Perda:
        sales_agent  total_opportunities  lost_opportunities  loss_rate
20      Moses Frase                  260                  88  33.846154
23  Rosalina Dieter                  160                  54  33.750000
3    Cecily Lampkin                  203                  67  33.004926
19  Maureen Marcano                  285                  94  32.982456
11    Hayden Neloms                  202                  61  30

# --- Análise da Taxa de Perda por Gerente ---

In [132]:
print("\n--- Análise da Taxa de Perda por Gerente ---")

# Calcular a contagem de oportunidades por gerente e a contagem de oportunidades perdidas por gerente
manager_analysis = df_eda_consolidated.groupby('manager')['target'].agg(
    total_opportunities='count',
    lost_opportunities=lambda x: (x == 1).sum()
).reset_index()

# Calcular a taxa de perda por gerente
manager_analysis['loss_rate'] = (manager_analysis['lost_opportunities'] / manager_analysis['total_opportunities']) * 100

# Ordenar para visualização
manager_analysis = manager_analysis.sort_values(by='loss_rate', ascending=False)

print(manager_analysis)

# Plot com Plotly: Taxa de Perda por Gerente
fig_manager = px.bar(manager_analysis,
                     x='manager',
                     y='loss_rate',
                     title='Taxa de Perda de Oportunidades por Gerente',
                     labels={'manager': 'Gerente', 'loss_rate': 'Taxa de Perda (%)'},
                     color='loss_rate',
                     color_continuous_scale="RdYlGn_r",
                     text_auto='.2f')

fig_manager.update_layout(xaxis_title="Gerente",
                          yaxis_title="Taxa de Perda (%)",
                          xaxis_tickangle=-45)
fig_manager.show()


--- Análise da Taxa de Perda por Gerente ---
            manager  total_opportunities  lost_opportunities  loss_rate
4     Rocco Neubert                 1327                 509  38.357197
3     Melvin Marxen                 1929                 726  37.636081
2  Dustin Brinkmann                 1583                 588  37.144662
0        Cara Losch                  964                 350  36.307054
1      Celia Rouche                 1296                 469  36.188272
5     Summer Sewald                 1701                 607  35.684891


# --- Análise da Taxa de Perda por Escritório Regional ---

In [133]:
print("\n--- Análise da Taxa de Perda por Escritório Regional ---")

# Calcular a contagem de oportunidades por escritório regional e a contagem de oportunidades perdidas por escritório regional
regional_analysis = df_eda_consolidated.groupby('regional_office')['target'].agg(
    total_opportunities='count',
    lost_opportunities=lambda x: (x == 1).sum()
).reset_index()

# Calcular a taxa de perda por escritório regional
regional_analysis['loss_rate'] = (regional_analysis['lost_opportunities'] / regional_analysis['total_opportunities']) * 100

# Ordenar para visualização
regional_analysis = regional_analysis.sort_values(by='loss_rate', ascending=False)

print(regional_analysis)

# Plot com Plotly: Taxa de Perda por Escritório Regional
fig_regional = px.bar(regional_analysis,
                      x='regional_office',
                      y='loss_rate',
                      title='Taxa de Perda de Oportunidades por Escritório Regional',
                      labels={'regional_office': 'Escritório Regional', 'loss_rate': 'Taxa de Perda (%)'},
                      color='loss_rate',
                      color_continuous_scale="RdYlGn_r", # Correção final aplicada aqui
                      text_auto='.2f')

fig_regional.update_layout(xaxis_title="Escritório Regional",
                           yaxis_title="Taxa de Perda (%)")
fig_regional.show()


--- Análise da Taxa de Perda por Escritório Regional ---
  regional_office  total_opportunities  lost_opportunities  loss_rate
1            East                 2291                 859  37.494544
0         Central                 3512                1314  37.414579
2            West                 2997                1076  35.902569


# --- 1. Duração da Oportunidade ---

In [135]:
print("\n--- Análise da Duração da Oportunidade ---")

# Filtrar oportunidades que possuem ambas as datas para calcular a duração
df_duration_analysis = df_eda_consolidated.dropna(subset=['engage_date', 'close_date', 'target']).copy()

# Calcular a duração em dias
df_duration_analysis['opportunity_duration_days'] = (df_duration_analysis['close_date'] - df_duration_analysis['engage_date']).dt.days

# Remover durações negativas ou zero (indica erro nos dados ou oportunidades fechadas no mesmo dia de engajamento)
df_duration_analysis = df_duration_analysis[df_duration_analysis['opportunity_duration_days'] > 0]

print("\nEstatísticas descritivas da Duração da Oportunidade (em dias):")
print(df_duration_analysis['opportunity_duration_days'].describe())

# Box Plot: Comparação da Duração para Oportunidades Perdidas vs. Não Perdidas
fig_duration = px.box(df_duration_analysis,
                      x='target',
                      y='opportunity_duration_days',
                      color='target',
                      title='Distribuição da Duração da Oportunidade (dias) para Perdidas vs. Não Perdidas',
                      labels={'target': 'Status da Oportunidade', 'opportunity_duration_days': 'Duração (dias)'},
                      color_discrete_map={0: 'lightgreen', 1: 'salmon'},
                      log_y=True) # Usar escala logarítmica se houver grande variação

fig_duration.update_layout(xaxis_title="0: Não Perdida | 1: Perdida")
fig_duration.show()


--- Análise da Duração da Oportunidade ---

Estatísticas descritivas da Duração da Oportunidade (em dias):
count    6711.000000
mean       47.985397
std        41.057665
min         1.000000
25%         8.000000
50%        45.000000
75%        85.000000
max       138.000000
Name: opportunity_duration_days, dtype: float64


In [136]:
# Calcular média da duração por status
print("\n--- Duração Média da Oportunidade (dias) por Status ---")
print(df_duration_analysis.groupby('target')['opportunity_duration_days'].mean())


--- Duração Média da Oportunidade (dias) por Status ---
target
0    51.781973
1    41.479175
Name: opportunity_duration_days, dtype: float64


# --- 2. Análise do Valor da Oportunidade (close_value) ---

In [137]:
print("\n--- Análise do Valor da Oportunidade (close_value) ---")

# Para esta análise, lidaremos com os NaNs da forma mais adequada.
# Se NaN em close_value significa "não houve valor porque a oportunidade não se concretizou (nem foi perdida)",
# então podemos preencher NaN com 0 para visualizar todas as oportunidades juntas.
# Já sabemos que para target=1, close_value é 0.0.
# O problema é com target=0 (ganha) e close_value=NaN.

# Vamos preencher NaNs em close_value com 0 para permitir a visualização comparativa completa.
# Isso trata os casos onde uma oportunidade GANHA (target=0) pode ter um close_value NaN (se existir).
df_value_analysis = df_eda_consolidated.copy()
df_value_analysis['close_value_filled'] = df_value_analysis['close_value'].fillna(0)

# Box Plot: Comparação do Valor para Oportunidades Perdidas vs. Não Perdidas

fig_value = px.box(df_value_analysis,
                   x='target',
                   y='close_value_filled',
                   color='target',
                   title='Distribuição do Valor da Oportunidade para Perdidas vs. Não Perdidas',
                   labels={'target': 'Status da Oportunidade', 'close_value_filled': 'Valor da Oportunidade'},
                   color_discrete_map={0: 'lightgreen', 1: 'salmon'})

fig_value.update_layout(xaxis_title="0: Não Perdida | 1: Perdida",
                        yaxis_title="Valor da Oportunidade")
fig_value.show()


--- Análise do Valor da Oportunidade (close_value) ---


In [138]:
# Histogramas para ver a distribuição mais claramente
fig_hist_won = px.histogram(df_value_analysis[df_value_analysis['target'] == 0],
                            x='close_value_filled',
                            nbins=50,
                            title='Distribuição do Valor das Oportunidades GANHAS',
                            labels={'close_value_filled': 'Valor Fechado'},
                            color_discrete_sequence=['lightgreen'])
fig_hist_won.update_layout(xaxis_title="Valor Fechado",
                           yaxis_title="Contagem de Oportunidades")
fig_hist_won.show()

# Para o target=1, o histograma seria apenas uma barra em 0, o que já sabemos.
# Podemos adicionar uma média para a distribuição das oportunidades ganhas, se for útil.
print("\n--- Estatísticas Descritivas do Valor da Oportunidade para Oportunidades GANHAS (target=0) ---")
print(df_value_analysis[df_value_analysis['target'] == 0]['close_value_filled'].describe())

# Média do valor para oportunidades ganhas vs. perdidas (que é 0)
print("\n--- Valor Médio da Oportunidade por Status ---")
print(df_value_analysis.groupby('target')['close_value_filled'].mean())


--- Estatísticas Descritivas do Valor da Oportunidade para Oportunidades GANHAS (target=0) ---
count     5551.000000
mean      1802.474149
std       2439.150665
min          0.000000
25%         48.000000
50%        579.000000
75%       3618.500000
max      30288.000000
Name: close_value_filled, dtype: float64

--- Valor Médio da Oportunidade por Status ---
target
0    1802.474149
1    2424.045648
Name: close_value_filled, dtype: float64


In [139]:
output_dir = '../data/processed'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Salvar o DataFrame consolidado na pasta 'processed'
df_eda_consolidated.to_csv(os.path.join(output_dir, 'df_eda_consolidated.csv'), index=False)
print(f"DataFrame df_eda_consolidated salvo em '{os.path.join(output_dir, 'df_eda_consolidated.csv')}'")

DataFrame df_eda_consolidated salvo em '../data/processed\df_eda_consolidated.csv'
